In [2]:
import requests
from datetime import datetime, timedelta
import pandas as pd

# Función para generar las fechas en el rango
def generar_fechas(inicio, fin):
    fecha_inicio = datetime.strptime(inicio, "%Y-%m-%d")
    fecha_fin = datetime.strptime(fin, "%Y-%m-%d")
    delta = timedelta(days=1)
    fechas = []
    while fecha_inicio <= fecha_fin:
        fechas.append(fecha_inicio.strftime("%Y-%m-%d"))
        fecha_inicio += delta
    return fechas

# Rango de fechas deseado
fecha_inicio = "2025-04-01"
fecha_fin = "2025-04-04"
fechas_esperadas = generar_fechas(fecha_inicio, fecha_fin)

# Lista para almacenar los resultados
resultados = []

# Consultar la API para cada fecha
for fecha in fechas_esperadas:
    url = f"https://free.e-api.net.pe/tipo-cambio/{fecha}.json"
    try:
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            resultados.append({
                "fecha": data.get("fecha"),
                "sunat": data.get("sunat"),
                "compra": data.get("compra"),
                "venta": data.get("venta")
            })
        else:
            print(f"{fecha}: Error {response.status_code}")
    except Exception as e:
        print(f"{fecha}: Error al consultar la API - {e}")

# Crear DataFrame con los resultados
df = pd.DataFrame(resultados)

# Convertir la columna 'fecha' a tipo datetime
df["fecha"] = pd.to_datetime(df["fecha"])

# Mostrar el DataFrame
print("\nDataFrame de tipo de cambio:")
print(df)

# -------------------------------
# Indicadores de calidad de datos
# -------------------------------
print("\n--- Indicadores de calidad de datos ---")

# 1. Verificar valores nulos
print("\nValores nulos por columna:")
print(df.isnull().sum())

# 2. Fechas duplicadas
duplicadas = df["fecha"].duplicated().sum()
print(f"\nFechas duplicadas: {duplicadas}")

# 3. Rango de tipo de cambio (valores fuera de límites esperados)
fuera_de_rango = df[
    (df["compra"] < 2) | (df["compra"] > 6) |
    (df["venta"] < 2)  | (df["venta"] > 6) |
    (df["sunat"] < 2)  | (df["sunat"] > 6)
]
print(f"\nFilas con valores fuera del rango [2, 6]: {len(fuera_de_rango)}")
if not fuera_de_rango.empty:
    print(fuera_de_rango)

# 4. Días faltantes (comparar con rango original)
fechas_obtenidas = df["fecha"].dt.strftime("%Y-%m-%d").tolist()
faltantes = set(fechas_esperadas) - set(fechas_obtenidas)
print(f"\nFechas faltantes: {faltantes if faltantes else 'Ninguna'}")




DataFrame de tipo de cambio:
       fecha   sunat  compra  venta
0 2025-04-01  3.6490   3.647  3.651
1 2025-04-02  3.6685   3.660  3.677
2 2025-04-03  3.6715   3.667  3.676
3 2025-04-04  3.6755   3.671  3.680

--- Indicadores de calidad de datos ---

Valores nulos por columna:
fecha     0
sunat     0
compra    0
venta     0
dtype: int64

Fechas duplicadas: 0

Filas con valores fuera del rango [2, 6]: 0

Fechas faltantes: Ninguna


# JUSTIFICACION DE LOS INDICADORES DE CALIDAD ###

### Valores nulos por columna: 
Esto indicará si hay valores faltantes en alguna fecha
### Fechas duplicadas: 
Esto evitara fallas al referenciar el tipo de cambio en alguna fecha
### Filas con valores fuera del rango [2, 6]: 
Esto asegura que el tipo de cambio este en un rango razonable, si no es asi, puede tratarse de un error en la extraccion de datos
### Fechas faltantes: 
Esto asegura tener el tipo de cambio en todas las fechas esperadas en la consulta.